In [3]:
import os
import json
import gzip
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import argparse

In [65]:
log_dir = "logs/PersONAL/easy"
# log_dir = "logs/PersONAL/medium"

data_content_dir = "data/datasets/PersONAL/val/easy/content"

len(os.listdir(log_dir))

31

In [86]:
scene_id, episode_id = "q5QZSEeHe5g", "518"

scene_info_path = os.path.join(data_content_dir, f"{scene_id}.json.gz")
print(f"Scene : {scene_id}, Episode : {episode_id}")

with gzip.open(scene_info_path, "r") as f_scene:
    scene_info = json.load(f_scene)

ids = []
for ep in scene_info["episodes"]:

    print(ep["object_id"], ep["object_category"])

    ids.append(ep["episode_id"])

    if ep["episode_id"] == episode_id:
        print(f"Found!")
        break

    # if ep["object_id"] == "blanket_207":
    #     print(f"Found thru obj ID!")
    #     break

Scene : q5QZSEeHe5g, Episode : 518
bathrobe_27 bathrobe
bed_142 bed
tv_113 tv
bathrobe_27 bathrobe
toilet_20 toilet
bathrobe_27 bathrobe
boxes_1317 box
board_1377 board
bed_1178 bed
chair_1450 chair
board_1467 board
bed_1178 bed


In [94]:
for goal in scene_info["goals_by_category"]:

    goal_infos = scene_info["goals_by_category"][goal]

    for goal_info in goal_infos:
        print(f"{goal_info['object_name']}, {goal_info['object_category']}, {goal_info['object_name']}")
    # break

bathrobe_27, bathrobe, bathrobe_27
bed_142, bed, bed_142
bed_1178, bed, bed_1178
blanket_272, blanket, blanket_272
tv_113, tv, tv_113
tv_1182, tv, tv_1182
toilet_20, toilet, toilet_20
window shutter_259, window shutter, window shutter_259
boxes_1317, box, boxes_1317
board_1377, board, board_1377
board_1467, board, board_1467
container_1389, container, container_1389
chair_1450, chair, chair_1450


In [92]:
goal_info

[{'position': [-0.58496, 4.41101, -3.23173],
  'radius': None,
  'object_id': 27,
  'object_name': 'bathrobe_27',
  'object_name_id': None,
  'object_category': 'bathrobe',
  'floor_id': '2',
  'room_id': None,
  'room_name': None,
  'view_points': [{'agent_state': {'position': [-0.2087, 3.36917, -2.95582],
     'rotation': [0.0, 0.45202, -0.0, 0.89201]},
    'iou': 0.5719},
   {'agent_state': {'position': [0.0413, 3.35388, -3.20582],
     'rotation': [0.0, 0.69234, -0.0, 0.72158]},
    'iou': 0.42799},
   {'agent_state': {'position': [0.0413, 3.35388, -2.95582],
     'rotation': [0.0, 0.54627, -0.0, 0.83761]},
    'iou': 0.3825},
   {'agent_state': {'position': [-0.4587, 3.38754, -2.70582],
     'rotation': [0.0, 0.11754, -0.0, 0.99307]},
    'iou': 0.36852},
   {'agent_state': {'position': [-0.2087, 3.37698, -2.70582],
     'rotation': [0.0, 0.30554, -0.0, 0.95218]},
    'iou': 0.3547},
   {'agent_state': {'position': [0.0413, 3.35745, -2.70582],
     'rotation': [0.0, 0.42244, -0.0,

In [90]:
scene_info["goals_by_category"][goal]

[{'position': [-0.58496, 4.41101, -3.23173],
  'radius': None,
  'object_id': 27,
  'object_name': 'bathrobe_27',
  'object_name_id': None,
  'object_category': 'bathrobe',
  'floor_id': '2',
  'room_id': None,
  'room_name': None,
  'view_points': [{'agent_state': {'position': [-0.2087, 3.36917, -2.95582],
     'rotation': [0.0, 0.45202, -0.0, 0.89201]},
    'iou': 0.5719},
   {'agent_state': {'position': [0.0413, 3.35388, -3.20582],
     'rotation': [0.0, 0.69234, -0.0, 0.72158]},
    'iou': 0.42799},
   {'agent_state': {'position': [0.0413, 3.35388, -2.95582],
     'rotation': [0.0, 0.54627, -0.0, 0.83761]},
    'iou': 0.3825},
   {'agent_state': {'position': [-0.4587, 3.38754, -2.70582],
     'rotation': [0.0, 0.11754, -0.0, 0.99307]},
    'iou': 0.36852},
   {'agent_state': {'position': [-0.2087, 3.37698, -2.70582],
     'rotation': [0.0, 0.30554, -0.0, 0.95218]},
    'iou': 0.3547},
   {'agent_state': {'position': [0.0413, 3.35745, -2.70582],
     'rotation': [0.0, 0.42244, -0.0,

In [95]:
dist_btw_pts = lambda pt_1, pt_2: np.linalg.norm(pt_1 - pt_2)

def get_dtg(final_pos, obj_cat, obj_name,
            scene_name, scene_info,
            to_vw_pt = True):

    obj_pos, obj_view_pts = [], []

    #Obtain the position of the target object and viewpoints
    goal_key = f"{scene_name}.basis.glb_{obj_cat}"
    for goal_instance in scene_info["goals_by_category"][goal_key]:

        if goal_instance["object_name"] != obj_name: continue   #Skip if not instance

        curr_view_pts = goal_instance["view_points"]
        curr_view_pts = [pt["agent_state"]["position"] for pt in curr_view_pts]

        obj_pos.append(goal_instance["position"])
        obj_view_pts.append(curr_view_pts)

    obj_pos, obj_view_pts = np.array(obj_pos), np.vstack(obj_view_pts)
    obj_pos, obj_view_pts = obj_pos[:, [0, 2]], obj_view_pts[:, [0, 2]]

    #Calculate distance to goal
    if to_vw_pt:
        view_pt_dists = np.apply_along_axis(lambda v: dist_btw_pts(v, final_pos), axis=1, arr=obj_view_pts)
        dtg = min(view_pt_dists)

    else:
        obj_dists = np.apply_along_axis(lambda v: dist_btw_pts(v, final_pos), axis=1, arr=obj_pos)
        dtg = min(obj_dists)
    
    return dtg
    

In [80]:
dist_thresh = 1
dtg_to_vw = True

In [97]:
success = 0
spl = 0

keys = []
spl_vals = []
dtg_vals = []

num_eps = 0

for f_name in tqdm(os.listdir(log_dir)):

    if not f_name.endswith(".txt"): continue

    #Trajetory
    f_path = os.path.join(log_dir, f_name)
    with open(f_path, "r") as f:
        traj = f.readlines()

    traj = [[float(s_sub) for s_sub in s.strip().split(",")] for s in traj]
    traj = np.array(traj)
    traj = traj[:, [1, 3]]

    deltas = traj[1:, :] - traj[:-1, :]
    traj_dist = np.linalg.norm(deltas, axis=1).sum()

    #Ground Truth
    scene_id, episode_id = f_name.split("_")[-1][:-4], f_name.split("_")[0]
    scene_info_path = os.path.join(data_content_dir, f"{scene_id}.json.gz")
    print(f"Scene : {scene_id}, Episode : {episode_id}")

    with gzip.open(scene_info_path, "r") as f_scene:
        scene_info = json.load(f_scene)

    #Obtain episode info
    found_ep = False
    for ep_info in scene_info["episodes"]:
        if ep_info["episode_id"] == episode_id:
            found_ep = True
            break

    assert found_ep

    #Distance to Goal (nearest viewpoint)
    dtg = get_dtg(final_pos = traj[-1],
                    obj_cat = ep_info["object_category"],
                    obj_name = ep_info["object_id"],
                    scene_name = scene_id,
                    scene_info = scene_info,
                    to_vw_pt = True)

    dtg_vals.append(dtg)
    if dtg <= dist_thresh:

        success += 1
    
        #Success Path Length (SPL)
        geo_dist = ep_info["info"]["geodesic_distance"]
        spl_ratio = geo_dist / max(geo_dist, traj_dist)
        spl_vals.append(spl_ratio)

    num_eps += 1
    # break

    


print(f"Success Rate : {success}/{num_eps} -> {(success/num_eps) * 100}")
print(f"SPL : {sum(spl_vals)/num_eps}")
print(f"DTG : {sum(dtg_vals)/num_eps}")

  3%|▎         | 16/600 [00:00<00:04, 143.72it/s]

Scene : 4ok3usBNeis, Episode : 205
Scene : bxsVRursffK, Episode : 485
Scene : GLAQ4DNUx5U, Episode : 135
Scene : wcojb4TFT35, Episode : 495
Scene : 5cdEh9F2hJL, Episode : 360
Scene : ziup5kvtCCR, Episode : 423
Scene : LT9Jq6dN3Ea, Episode : 762
Scene : cvZr5TUy5C5, Episode : 549
Scene : h1zeeAwLh9Z, Episode : 155
Scene : DYehNKdT76V, Episode : 182
Scene : a8BtkwhxdRV, Episode : 642
Scene : cvZr5TUy5C5, Episode : 551
Scene : BAbdmeyTvMZ, Episode : 4
Scene : LT9Jq6dN3Ea, Episode : 746
Scene : DYehNKdT76V, Episode : 184
Scene : p53SfW6mjZe, Episode : 591
Scene : mv2HUxq3B53, Episode : 389
Scene : CrMo8WxCyVb, Episode : 667
Scene : qyAac8rV8Zk, Episode : 699
Scene : DYehNKdT76V, Episode : 180
Scene : q5QZSEeHe5g, Episode : 532
Scene : q5QZSEeHe5g, Episode : 530
Scene : DYehNKdT76V, Episode : 183
Scene : VBzV5z6i1WS, Episode : 463
Scene : 4ok3usBNeis, Episode : 209
Scene : VBzV5z6i1WS, Episode : 458
Scene : q5QZSEeHe5g, Episode : 534
Scene : cvZr5TUy5C5, Episode : 573
Scene : a8BtkwhxdRV, E

 11%|█         | 65/600 [00:00<00:02, 190.73it/s]

Scene : a8BtkwhxdRV, Episode : 634
Scene : zt1RVoi7PcG, Episode : 167
Scene : svBbv1Pavdk, Episode : 281
Scene : cvZr5TUy5C5, Episode : 546
Scene : DYehNKdT76V, Episode : 178
Scene : wcojb4TFT35, Episode : 522
Scene : VBzV5z6i1WS, Episode : 462
Scene : zt1RVoi7PcG, Episode : 162
Scene : TEEsavR23oF, Episode : 246
Scene : QaLdnwvtxbs, Episode : 784
Scene : cvZr5TUy5C5, Episode : 550
Scene : qyAac8rV8Zk, Episode : 698
Scene : zt1RVoi7PcG, Episode : 170
Scene : cvZr5TUy5C5, Episode : 566
Scene : Nfvxx8J5NCo, Episode : 722
Scene : BAbdmeyTvMZ, Episode : 14
Scene : cvZr5TUy5C5, Episode : 552
Scene : DYehNKdT76V, Episode : 188
Scene : 6s7QHgap2fW, Episode : 345
Scene : MHPLjHsuG27, Episode : 218
Scene : GLAQ4DNUx5U, Episode : 106
Scene : y9hTuugGdiq, Episode : 44
Scene : h1zeeAwLh9Z, Episode : 137
Scene : wcojb4TFT35, Episode : 523
Scene : a8BtkwhxdRV, Episode : 630
Scene : VBzV5z6i1WS, Episode : 447
Scene : QaLdnwvtxbs, Episode : 803
Scene : p53SfW6mjZe, Episode : 577
Scene : Dd4bFSTQ8gi, E

 19%|█▉        | 115/600 [00:00<00:02, 202.32it/s]

...

Scene : bCPU9suPUw9, Episode : 606
Scene : 6s7QHgap2fW, Episode : 342
Scene : wcojb4TFT35, Episode : 505
Scene : bxsVRursffK, Episode : 490
Scene : XB4GS9ShBRE, Episode : 396
Scene : 5cdEh9F2hJL, Episode : 362
Scene : TEEsavR23oF, Episode : 269
Scene : 5cdEh9F2hJL, Episode : 353
Scene : Nfvxx8J5NCo, Episode : 726
Scene : 6s7QHgap2fW, Episode : 341
Scene : MHPLjHsuG27, Episode : 235
Scene : Nfvxx8J5NCo, Episode : 709
Scene : qyAac8rV8Zk, Episode : 696
Scene : 6s7QHgap2fW, Episode : 333
Scene : 6s7QHgap2fW, Episode : 331
Scene : 6s7QHgap2fW, Episode : 349
Scene : q3zU7Yy5E5s, Episode : 69
Scene : 5cdEh9F2hJL, Episode : 356
Scene : DYehNKdT76V, Episode : 174
Scene : y9hTuugGdiq, Episode : 32
Scene : QaLdnwvtxbs, Episode : 804
Scene : QaLdnwvtxbs, Episode : 795
Scene : BAbdmeyTvMZ, Episode : 10
Scene : zt1RVoi7PcG, Episode : 163
Scene : Dd4bFSTQ8gi, Episode : 90
Scene : GLAQ4DNUx5U, Episode : 125
Scene : svBbv1Pavdk, Episode : 279
Scene : XB4GS9ShBRE, Episode : 415
Scene : qyAac8rV8Zk, Epi

 94%|█████████▎| 561/600 [00:02<00:00, 216.96it/s]

Scene : mL8ThkuaVTM, Episode : 294
Scene : XB4GS9ShBRE, Episode : 394
Scene : MHPLjHsuG27, Episode : 219
Scene : p53SfW6mjZe, Episode : 588
Scene : 6s7QHgap2fW, Episode : 335
Scene : MHPLjHsuG27, Episode : 233
Scene : wcojb4TFT35, Episode : 493
Scene : GLAQ4DNUx5U, Episode : 108
Scene : ziup5kvtCCR, Episode : 421
Scene : CrMo8WxCyVb, Episode : 663
Scene : TEEsavR23oF, Episode : 255
Scene : QaLdnwvtxbs, Episode : 787
Scene : LT9Jq6dN3Ea, Episode : 767
Scene : 6s7QHgap2fW, Episode : 346
Scene : p53SfW6mjZe, Episode : 584
Scene : XB4GS9ShBRE, Episode : 411
Scene : qyAac8rV8Zk, Episode : 707
Scene : QaLdnwvtxbs, Episode : 802
Scene : qyAac8rV8Zk, Episode : 686
Scene : wcojb4TFT35, Episode : 494
Scene : QaLdnwvtxbs, Episode : 799
Scene : QaLdnwvtxbs, Episode : 786
Scene : 4ok3usBNeis, Episode : 199
Scene : qyAac8rV8Zk, Episode : 689
Scene : GLAQ4DNUx5U, Episode : 136
Scene : 4ok3usBNeis, Episode : 213
Scene : cvZr5TUy5C5, Episode : 569
Scene : DYehNKdT76V, Episode : 173
Scene : mv2HUxq3B53,

100%|██████████| 600/600 [00:02<00:00, 207.74it/s]

Scene : p53SfW6mjZe, Episode : 596
Scene : QaLdnwvtxbs, Episode : 793
Scene : TEEsavR23oF, Episode : 240
Scene : qyAac8rV8Zk, Episode : 694
Scene : 4ok3usBNeis, Episode : 197
Scene : y9hTuugGdiq, Episode : 29
Scene : Nfvxx8J5NCo, Episode : 721
Scene : p53SfW6mjZe, Episode : 592
Scene : Dd4bFSTQ8gi, Episode : 85
Scene : TEEsavR23oF, Episode : 263
Scene : QaLdnwvtxbs, Episode : 805
Scene : VBzV5z6i1WS, Episode : 438
Scene : mv2HUxq3B53, Episode : 380
Scene : LT9Jq6dN3Ea, Episode : 744
Scene : 6s7QHgap2fW, Episode : 317
Scene : q5QZSEeHe5g, Episode : 526
Scene : 4ok3usBNeis, Episode : 200
Scene : ziup5kvtCCR, Episode : 437
Scene : wcojb4TFT35, Episode : 500
Scene : GLAQ4DNUx5U, Episode : 126
Scene : LT9Jq6dN3Ea, Episode : 763
Scene : DYehNKdT76V, Episode : 175
Scene : MHPLjHsuG27, Episode : 220
Scene : qyAac8rV8Zk, Episode : 672
Scene : cvZr5TUy5C5, Episode : 570
Scene : y9hTuugGdiq, Episode : 48
Scene : q5QZSEeHe5g, Episode : 538
Scene : a8BtkwhxdRV, Episode : 624
Scene : mL8ThkuaVTM, Ep

In [70]:
for ep in scene_info["episodes"]:

    if ep["episode_id"] == episode_id:
        print(f"Found!")
        break

Found!


In [71]:
ep

{'episode_id': '423',
 'scene_id': 'hm3d_v0.2/val/00848-ziup5kvtCCR/ziup5kvtCCR.basis.glb',
 'scene_dataset_config': './data/scene_datasets/hm3d_v0.2/hm3d_annotated_basis.scene_dataset_config.json',
 'object_category': 'washer-dryer',
 'object_id': 'washer-dryer_135',
 'description': ['stacked washer-dryer located under the white cabinet.',
  '',
  ''],
 'owner': 'Eliana',
 'floor_id': '0',
 'summary': 'On the ground floor of this home, there are several distinct and well-described items, each owned by a different individual. In the laundry room, Eliana owns a stacked washer-dryer located under the white cabinet. Moving to the living room, Lorenzo owns a sideboard with a flat screen TV on it, positioned in front of the couch. In bedroom 1, Christian owns a TV that is mounted on the wall, situated next to a mirror and above a cabinet, close to the bed. Finally, in the master bedroom bathroom, Ayden owns the sink cabinet next to the bathtub, a white marble cabinet with a mirror on top. E

In [87]:
scene_info["goals_by_category"]

{'q5QZSEeHe5g.basis.glb_bathrobe': [{'position': [-0.58496, 4.41101, -3.23173],
   'radius': None,
   'object_id': 27,
   'object_name': 'bathrobe_27',
   'object_name_id': None,
   'object_category': 'bathrobe',
   'floor_id': '2',
   'room_id': None,
   'room_name': None,
   'view_points': [{'agent_state': {'position': [-0.2087, 3.36917, -2.95582],
      'rotation': [0.0, 0.45202, -0.0, 0.89201]},
     'iou': 0.5719},
    {'agent_state': {'position': [0.0413, 3.35388, -3.20582],
      'rotation': [0.0, 0.69234, -0.0, 0.72158]},
     'iou': 0.42799},
    {'agent_state': {'position': [0.0413, 3.35388, -2.95582],
      'rotation': [0.0, 0.54627, -0.0, 0.83761]},
     'iou': 0.3825},
    {'agent_state': {'position': [-0.4587, 3.38754, -2.70582],
      'rotation': [0.0, 0.11754, -0.0, 0.99307]},
     'iou': 0.36852},
    {'agent_state': {'position': [-0.2087, 3.37698, -2.70582],
      'rotation': [0.0, 0.30554, -0.0, 0.95218]},
     'iou': 0.3547},
    {'agent_state': {'position': [0.0413